In [5]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

# 1. Configuración idéntica a tus ejemplos habituales
packages = [
    "org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.0", # Ajustado a tu versión
    "org.apache.hadoop:hadoop-aws:3.3.4",
    "com.amazonaws:aws-java-sdk-bundle:1.12.262"
]



In [13]:
spark = SparkSession.builder \
    .appName("HumidityStreamingFinal_Snapshot6") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.0") \
    .config("spark.hadoop.fs.s3a.endpoint", "http://minio:9000") \
    .config("spark.hadoop.fs.s3a.access.key", "minioadmin") \
    .config("spark.hadoop.fs.s3a.secret.key", "minioadmin") \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .getOrCreate()

In [14]:

schema = StructType([
    StructField("sensor_id", StringType(), True),
    StructField("humidity", DoubleType(), True),
    StructField("timestamp", StringType(), True),
    StructField("location", StringType(), True)
])

In [19]:

df_kafka = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "kafka:29092") \
    .option("subscribe", "humidity-sensors") \
    .option("startingOffsets", "earliest") \
    .option("failOnDataLoss", "false") \
    .load()

In [21]:
parsed_df = df_kafka.select(
    from_json(col("value").cast("string"), schema).alias("data")
).select("data.*")

monitor = parsed_df.agg(
    avg("humidity").alias("Promedio_Humedad_Streaming"),
    count("*").alias("Total_Lecturas_Recibidas")
)

In [22]:
query = monitor.writeStream \
    .outputMode("complete") \
    .format("memory") \
    .queryName("tabla_humedad_final") \
    .start()

In [23]:
import time
print("Esperando a que los datos fluyan...")
time.sleep(10)
spark.sql("select * from tabla_humedad_final").show()

🔄 Intento 1/5 - Esperando flujo de datos...
+--------------------------+------------------------+
|Promedio_Humedad_Streaming|Total_Lecturas_Recibidas|
+--------------------------+------------------------+
|         69.59070619309112|                   14387|
+--------------------------+------------------------+

🔄 Intento 2/5 - Esperando flujo de datos...
+--------------------------+------------------------+
|Promedio_Humedad_Streaming|Total_Lecturas_Recibidas|
+--------------------------+------------------------+
|          69.5873319897145|                   14389|
+--------------------------+------------------------+

🔄 Intento 3/5 - Esperando flujo de datos...
+--------------------------+------------------------+
|Promedio_Humedad_Streaming|Total_Lecturas_Recibidas|
+--------------------------+------------------------+
|          69.5841248002225|                   14391|
+--------------------------+------------------------+

🔄 Intento 4/5 - Esperando flujo de datos...
+----------